In [1]:

import torch
import os
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
import random

# Set the seed for reproducibility
SEED = 42
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)  # Apply the seed

In [ ]:
# Define the categories and the paths to datasets
checkpoint_path = r'C:\Users\Josh\Desktop\CUDA\52.41.pth'

BATCH_SIZE = 32
INPUT_SIZE = (256, 256)

def count_files_in_directory(directory):
    total_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
    return total_files

main_data_dir = r"C:\Users\Josh\Desktop\CUDA\splitdata"
train_dir = os.path.join(main_data_dir, "train")
val_dir = os.path.join(main_data_dir, "val")
test_dir = os.path.join(main_data_dir, "test")

train_files = count_files_in_directory(train_dir)
val_files = count_files_in_directory(val_dir)
test_files = count_files_in_directory(test_dir)

print(f"Training Dataset: {train_files}")
print(f"Validation Dataset: {val_files}")
print(f"Test Dataset: {test_files}")

In [ ]:

# Load the training dataset to calculate mean and std, and get class labels
train_dataset = datasets.ImageFolder(root=train_dir)
class_n = list(train_dataset.class_to_idx.keys())  # Automatically retrieves class names from folders
print("Class to label mapping:", train_dataset.class_to_idx)

In [4]:
MEAN = (0.5, 0.5, 0.5)
STD = (0.5, 0.5, 0.5)

transform_train = transforms.Compose([
    # Resize to standardize input dimensions
    transforms.Resize(INPUT_SIZE),
    transforms.CenterCrop((224, 224)), 

    # Apply segmentation after resizing
    # KMeansSegmentation(n_clusters=10, p=0.3),

    # Slight rotations with fill to avoid black edges
    transforms.RandomApply(
        [transforms.RandomRotation(degrees=(0, 20))], p=0.25
    ),
    transforms.RandomApply(
        [transforms.RandomRotation(degrees=(0, 10))], p=0.25
    ),
    transforms.RandomApply(
        [transforms.RandomRotation(degrees=(-20, 0))], p=0.25
    ),
    transforms.RandomApply(
        [transforms.RandomRotation(degrees=(-10, 0))], p=0.25
    ),

  
    # # Subtle affine transformations
    # transforms.RandomApply([
    #     transforms.RandomAffine(degrees=5, translate=(0.02, 0.02), shear=2)
    # ], p=0.3),

    # # Perspective distortion
    # transforms.RandomApply([
    #     transforms.RandomPerspective(distortion_scale=0.05, p=0.2)
    # ], p=0.2),

    # Minor Gaussian blur
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))
    ], p=0.4),

    # Color and grayscale variations
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.2, contrast=0.2)
    ], p=0.7),  
    
    transforms.RandomApply([
        transforms.RandomGrayscale(p=1.0)
    ], p=0.2),  

    # Horizontal flip
    transforms.RandomApply([transforms.RandomHorizontalFlip()], p=0.5),

    # Adjust sharpness slightly
    transforms.RandomApply([
        transforms.RandomAdjustSharpness(sharpness_factor=1.5)
    ], p=0.3),

    # Convert to tensor and normalize
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])



transform_val_test = transforms.Compose([
    transforms.Resize(INPUT_SIZE),
    transforms.CenterCrop((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

# Load the datasets with the new transforms
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform_train)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform_val_test)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform_val_test)


# Create DataLoaders for each dataset
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
# Get a batch of images and labels from the DataLoader
images, labels = next(iter(train_loader))
print(images.shape)  


In [ ]:

# Function to unnormalize the image for visualization
def unnormalize(image, mean, std):
    # Convert the tensor to a NumPy array and transpose dimensions to (H, W, C)
    image = image.numpy().transpose((1, 2, 0))  
    
    # Unnormalize by reversing the mean and std normalization
    image = (image * std) + mean  
    
    # Clip values to be between 0 and 1 for valid image display
    image = np.clip(image, 0, 1)  
    return image


# Visualize a batch of images from the train_loader
def visualize_loader(loader, mean, std, class_names, num_images=6):
    # Get a batch of images
    data_iter = iter(loader)
    images, labels = next(data_iter)  

    # Plot the images
    plt.figure(figsize=(12, 8))
    for i in range(num_images):
        plt.subplot(2, 3, i+1)
        image = unnormalize(images[i], mean, std)  
        plt.imshow(image)
        plt.title(f"Class: {class_names[labels[i]]}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Use this function to visualize a batch of images
visualize_loader(train_loader, mean=MEAN, std=STD, class_names=class_n)

In [ ]:


class AlexNetFC6(nn.Module):
    def __init__(self):
        super(AlexNetFC6, self).__init__()
        alexnet = models.alexnet(pretrained=False)
        self.features = alexnet.features
        self.pooling = nn.AdaptiveAvgPool2d((6, 6))
        self.fc6 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pooling(x) 
        x = torch.flatten(x, 1)
        x = self.fc6(x)
        return x



alexnet_fc6 = AlexNetFC6()
alexnet_fc6.load_state_dict(torch.load(checkpoint_path), strict=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet_fc6.to(device)
alexnet_fc6.eval()  




In [8]:
def extract_features(data_loader, model):
    features = []
    labels = []
    
    with torch.no_grad():  # Disable gradient computation
        for images, target_labels in data_loader:
            images = images.to(device)
            
            # Extract features using the custom AlexNet model (up to fc6)
            output_features = model(images)
            
            # Append extracted features and labels to the list
            features.append(output_features.cpu().numpy())  # Keeping this as CPU NumPy
            labels.append(target_labels.cpu().numpy())
    
    # Concatenate features and labels from all batches
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels


In [9]:
# Extract features from the datasets
train_features, y_train = extract_features(train_loader, alexnet_fc6)
val_features, y_val = extract_features(val_loader, alexnet_fc6)
test_features, y_test = extract_features(test_loader, alexnet_fc6)


In [ ]:
import joblib
from sklearn.preprocessing import StandardScaler

# Feature Scaling
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)
test_features_scaled = scaler.transform(test_features)

# Save the scaler model for later use
joblib.dump(scaler, 'scaler.joblib')

# Print shapes to confirm dimensions remain the same
print(f"Shape of train_features after scaling: {train_features_scaled.shape}")
print(f"Shape of val_features after scaling: {val_features_scaled.shape}")
print(f"Shape of test_features after scaling: {test_features_scaled.shape}")


In [ ]:
# Train XGBoost classifier
dtrain = xgb.DMatrix(train_features_scaled, label=y_train)
dval = xgb.DMatrix(val_features_scaled, label=y_val)
dtest = xgb.DMatrix(test_features_scaled, label=y_test)

# Verify if DMatrix creation was successful
print(f"Train DMatrix: {dtrain.num_row()} rows, {dtrain.num_col()} columns")
print(f"Validation DMatrix: {dval.num_row()} rows, {dval.num_col()} columns")
print(f"Test DMatrix: {dtest.num_row()} rows, {dtest.num_col()} columns")

# Check class distribution in training data
from collections import Counter
print(f"Training class distribution: {Counter(y_train)}")


In [ ]:
import optuna
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

# Define the objective function using xgb.train with DMatrix
def objective(trial):
    param = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'max_depth': trial.suggest_int('max_depth', 2, 6),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 0.9),
        'subsample': trial.suggest_uniform('subsample', 0.5, 0.9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'gamma': trial.suggest_uniform('gamma', 0, 5),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-2, 100),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-2, 100),
        'objective': 'multi:softmax',
        'num_class': len(class_n),
        'eval_metric': 'mlogloss',
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
    }

    # Use xgb.train with DMatrix
    evals = [(dtrain, 'train'), (dval, 'validation')]
    model = xgb.train(param, dtrain, num_boost_round=1000, evals=evals,
                      early_stopping_rounds=20, verbose_eval=True)
    
    val_preds = model.predict(dval)
    val_preds = val_preds.astype(int)  # Convert to int for compatibility with y_val
    val_accuracy = accuracy_score(y_val, val_preds)
    return val_accuracy

# Optimize hyperparameters with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Train a final model with the best hyperparameters using DMatrix
best_params = study.best_params
final_model = xgb.train(
    best_params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtrain, 'train'), (dval, 'validation')],
    early_stopping_rounds=20,
    verbose_eval=True
)



In [ ]:
# Predict and evaluate on the training set
train_predictions = final_model.predict(dtrain)
train_accuracy = accuracy_score(y_train, train_predictions)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

# Predict and evaluate on the validation set
val_predictions = final_model.predict(dval)
val_accuracy = accuracy_score(y_val, val_predictions)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

# Predict and evaluate on the test set
test_predictions = final_model.predict(dtest)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Generate and print classification reports for validation and test sets
val_classification_report = classification_report(y_val, val_predictions, target_names=class_n)
print("Validation Classification Report:\n", val_classification_report)

cm = confusion_matrix(y_val, val_predictions)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_n, yticklabels=class_n)

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

test_classification_report = classification_report(y_test, test_predictions, target_names=class_n)
print("Test Classification Report:\n", test_classification_report)

cm = confusion_matrix(y_test, test_predictions)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_n, yticklabels=class_n)

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
